## Imports

In [176]:
import time
from time import sleep
from tqdm import tqdm


#---------Selenium------------

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# generate random for time
import random

# for dates
from datetime import datetime, timedelta

# regex of course
import re

# pandas for dataframe
import pandas as pd


# chrome driver 
### https://chromedriver.chromium.org/

# FUNCTIONS

## Remove the Ads actually hide them..

In [2]:
def Count_remove_Ads(driver): 
    all_iframes = driver.find_elements_by_tag_name("iframe")
    if len(all_iframes) > 0:
#         print("Ad Found\n")
        driver.execute_script("""
            var elems = document.getElementsByTagName("iframe"); 
            for(var i = 0, max = elems.length; i < max; i++)
                 {
                     elems[i].hidden=true;
                 }
                              """)
#         print('Total Ads: ' + str(len(all_iframes)))
    else:
#         print('No frames found')
        pass

## Give all jobs in a single page

In [3]:
def All_Jobs_to_me(driver):
    
    #     All_Jobs_to_me "extract" the site, give all jobs in a single page



    #     create button for Click Περισσότερες Αγγελίες... 
    morejobs_btn = driver.find_element_by_id('ctl00_ContentplaceholderSubMenu_ASPxCallbackPanel1_ASPxDataView2_EPContainer')


    myflag=True
    i=1
    test=0
    while myflag :
        try:
            morejobs_btn.click()
  
            sleep(random.uniform(3, 4))
        except:
            sleep(random.uniform(2, 3))
            test+=1
#             print(f'tries {test}')
            if test>4:
                myflag=False
            continue

## Open the site and remove what ever appear

In [8]:
def opensite(myurl):

    # https://chromedriver.chromium.org/
    driverjob = webdriver.Chrome("C:\Program Files (x86)\chromedriver.exe")
    driverjob.get(myurl)
    


    # the first message appear!! 
    sleep(random.uniform(2, 3))
    for check in range(3):
        try:
            driverjob.find_element_by_xpath("//*[contains(text(), 'Οχι')]").click()
            break
        except:
            continue
            sleep(random.uniform(0,0.5))
    #---------------------------------------------------------------
    print(driverjob.title)
    #---------------------------------------------------------------

    
    # hide the ads!!
    sleep(random.uniform(2, 3))
    for check in range(3):
        try:
            Count_remove_Ads(driverjob)
            break
        except:            
            sleep(random.uniform(0,0.5))
            continue
    #---------------------------------------------------------------

    # agree for cookies 
    sleep(random.uniform(0.5,1))
    for check in range(3):
        try:
            driverjob.find_element_by_class_name("cc-compliance").click()
            break
        except:
            continue
            sleep(random.uniform(0.5,1))
    #---------------------------------------------------------------

    return driverjob

### Extract specific data from url

In [9]:
def job_analysis(driverjob):
    # the actually job in the site
    thejob=driverjob.find_element_by_id("ctl00_ContentplaceholderSubMenu_PanelLiveContent")
    info_of_job=thejob.find_elements_by_tag_name("a")
    specific=thejob.find_elements_by_class_name('col-md-6')
    specificjob=specific[0].find_elements_by_tag_name('p')
    specificjob1=specific[1].find_elements_by_tag_name('p')
    
    #------titleofjob------    
    titleofjob=thejob.find_element_by_tag_name("h1").text  
    #----------------------
    
    
    #----refernce_number---    
    refernce_number=int(info_of_job[0].text[10:])    
    #----------------------
    
    
    #---------city---------    
    city=info_of_job[1].text    
    #----------------------
    
    #------date_posted-----    
    date_posted=info_of_job[2].text
    if date_posted=='αναρτήθηκε σήμερα':    
        # get the date and time now
        now = datetime.now()
        date_posted = now.strftime("%d/%m/%Y")
    elif date_posted=='αναρτήθηκε χθές':
        # get the date and time now
        now = datetime.now()
        date_posted=(now - timedelta(hours=24)).strftime("%d/%m/%Y")
    else:
        try:
            Posted_days=int(re.findall(r'[0-9]+', date_posted)[0])
            # get the date and time now
            now = datetime.now()
            date_posted=(now + timedelta(days=Posted_days)).strftime("%d/%m/%Y")
        except:
            pass
    #----------------------
    
    #------expire_date-----   
    expire_date=info_of_job[4].text
    try:

        expire_days=int(re.findall(r'[0-9]+', expire_date)[0])
        # get the date and time now
        now = datetime.now()
        expire_date=(now + timedelta(days=expire_days)).strftime("%d/%m/%Y")
    except:
        pass
    #----------------------
    
    #---------views--------   
    views=int(info_of_job[5].text)
    #----------------------
    
    #------type_of_job-----   
    type_of_employment=specificjob1[0].text
    try:
        if type_of_employment[:18]=='ΤΥΠΟΣ ΑΠΑΣΧΟΛΗΣΗΣ:':
            type_of_employment=type_of_employment[19:]
    except:
        pass
    #----------------------
    
    #---------employment_sector--------   
    employment_sector=specificjob1[1].text
    try:
        if employment_sector[:19]=='ΚΛΑΔΟΣ ΑΠΑΣΧΟΛΗΣΗΣ:':
            employment_sector=employment_sector[20:]
    except:
        pass
    #----------------------------------
    
    #-------------publisher------------   
    publisher=specificjob[1].text
    try:
        if publisher[:18]=='ΔΗΜΟΣΙΟΠΟΙΗΣΗ ΑΠΟ:':
            publisher=publisher[19:]
    except:
        pass
    #----------------------------------
    
    #-------------category-------------   
    category=specificjob[0].text
    try:
        if category[:22]=='ΚΑΤΗΓΟΡΙΑ ΑΠΑΣΧΟΛΗΣΗΣ:':
            category=category[23:]
    except:
        pass
    #----------------------------------
    
    
    
    print(f"titleofjob={titleofjob}\nrefernce_number={refernce_number}\ncity={city}\ndate_posted={date_posted}\nexpire_date={expire_date}\nviews={views}\ntype_of_employment={type_of_employment}\nemployment_sector={employment_sector}\npublisher={publisher}\ncategory={category}")

    return (titleofjob, refernce_number, city, date_posted, expire_date, views, type_of_employment, employment_sector, publisher, category)


 # ------------------------------------------------------------------------------

### OPEN THE SITE

In [177]:
start = time.time()

url = "https://www.ergodotisi.com/"
driver=opensite(url)

end = time.time()
print(f"Runtime of the program is {end - start}")

Ergodotisi.com - Θέσεις εργασίας στην Κύπρο
Runtime of the program is 9.622676372528076


### Click Αναζητηση Εργασιας

In [6]:

sleep(random.uniform(0, 1))
SearchButton= WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[name='ctl00$ContentplaceholderMenu$ASPxButton1']")))
SearchButton.click()

### Put all job in the page

In [7]:
start = time.time()

All_Jobs_to_me(driver)

#sometimes appear !!
for check in range(3):
    try:
        driverjob.find_element_by_xpath("//*[contains(text(), 'Οχι')]").click()
        break
    except:        
        sleep(random.uniform(0,0.5))
        continue

end = time.time()
print(f"Runtime of the program is {end - start}")

Runtime of the program is 363.526620388031


### How many jobs have the site

In [12]:

tableJobs=driver.find_element_by_id("ctl00_ContentplaceholderSubMenu_ASPxCallbackPanel1_ASPxDataView2_ICell")
ListOfJobs=tableJobs.find_elements_by_tag_name("tr")
print(len(ListOfJobs))

1000


### number of job for each city

In [13]:
numJob=int(driver.find_element_by_id("ctl00_ContentplaceholderSubMenu_ASPxCallbackPanel1_resultscount").text)
print(numJob)

2170


### Get the links of jobs

In [14]:
# Get the links of jobs
linksjobs=[]
for job in tqdm(ListOfJobs):
    linksjobs.append(job.find_element_by_tag_name("a").get_attribute('href'))

100%|██████████| 1000/1000 [00:11<00:00, 85.74it/s]


### Create a df

In [179]:
cols = [
    'title_of_job', 'refernce_number', 'city', 
    'date_posted', 'expire_date', 'views',
    'type_of_employment', 'employment_sector', 
    'publisher', 'category'

]
df = pd.DataFrame(columns=cols)
df

,title_of_job,refernce_number,city,date_posted,expire_date,views,type_of_employment,employment_sector,publisher,category


### Extract specific data from each url

In [180]:
# extract specific data from each url

for Link in tqdm(linksjobs):
    try:
        job_driver=opensite(Link)
        titleofjob, refernce_number, city, date_posted, expire_date, views, type_of_employment, employment_sector, publisher, category=job_analysis(job_driver)
        job_driver.quit()
        sleep(random.uniform(2,3))
    except:
        pass
        #Complete the dataset
    #--------------------------
        temp = {
        'title_of_job': titleofjob,
        'refernce_number': refernce_number,
        'city':city, 
        'date_posted': date_posted,
        'expire_date': expire_date,
        'views': views,
        'type_of_employment': type_of_employment,
        'employment_sector':employment_sector, 
        'publisher': publisher,
        'category': category
        }
        df = df.append(temp, ignore_index=True, )

    
    

  0%|          | 0/1000 [00:00<?, ?it/s]

Γραμματέας - MESITIS REAL ESTATE AGENTS LTD
titleofjob=Γραμματέας
refernce_number=148792
city=Λευκωσία
date_posted=31/08/2021
expire_date=27/09/2021
views=469
type_of_employment=Πλήρης
employment_sector=Κτηματομεσιτικά γραφεία
publisher=Ιδιωτική Εταιρεία (LTD)
category=Γραμματειακά, Εξυπηρέτηση Πελατών, Λογιστές


  0%|          | 1/1000 [00:14<4:02:21, 14.56s/it]

Systems Engineer - NewCytech Business Solutions - Nicosia & Limassol - Logicom Public Ltd
titleofjob=Systems Engineer - NewCytech Business Solutions - Nicosia & Limassol
refernce_number=148797
city=Λεμεσός, Λευκωσία
date_posted=31/08/2021
expire_date=27/09/2021
views=159
type_of_employment=Πλήρης
employment_sector=Πληροφορική
publisher=Δημόσια Εταιρεία (PLC)
category=Πληροφορική


  0%|          | 2/1000 [00:29<4:03:05, 14.61s/it]

...

100%|█████████▉| 998/1000 [3:58:04<00:29, 14.52s/it]

Barista - Olebridge Investments ltd
titleofjob=Barista
refernce_number=147635
city=Λάρνακα
date_posted=09/09/2021
expire_date=18/09/2021
views=202
type_of_employment=Πλήρης
employment_sector=Εστιατόρια / Καφετέριες
publisher=Ιδιωτική Εταιρεία (LTD)
category=Τουριστικές Σπουδές, Υπάλληλοι HORECA, Baristas


100%|█████████▉| 999/1000 [3:58:18<00:14, 14.39s/it]

Sommelier - InterTaxAudit Auditors and Consultants Ltd
titleofjob=Sommelier
refernce_number=147575
city=Λεμεσός, Λευκωσία
date_posted=09/09/2021
expire_date=18/09/2021
views=262
type_of_employment=Πλήρης
employment_sector=Μη διαθέσιμο
publisher=Ιδιωτική Εταιρεία (LTD)
category=Υπάλληλοι HORECA


100%|██████████| 1000/1000 [3:58:32<00:00, 14.31s/it]


In [181]:
df

,title_of_job,refernce_number,city,date_posted,expire_date,views,type_of_employment,employment_sector,publisher,category
0,Γραμματέας,148792,Λευκωσία,31/08/2021,27/09/2021,469,Πλήρης,Κτηματομεσιτικά γραφεία,Ιδιωτική Εταιρεία (LTD),"Γραμματειακά, Εξυπηρέτηση Πελατών, Λογιστές"
1,Systems Engineer - NewCytech Business Solution...,148797,"Λεμεσός, Λευκωσία",31/08/2021,27/09/2021,159,Πλήρης,Πληροφορική,Δημόσια Εταιρεία (PLC),Πληροφορική
2,Senior Accountant,148790,Λεμεσός,31/08/2021,27/09/2021,311,Πλήρης,Λογιστικά / Ελεγκτικά γραφεία,Ιδιωτική Εταιρεία (LTD),"Λογιστές, Οικονομολόγοι, Χρηματοοικονομικά/Χρη..."
3,"Web Developer: Nicosia, Cyprus",148788,Λευκωσία,31/08/2021,27/09/2021,191,Πλήρης,Τηλεπικοινωνία / Internet,Ιδιωτική Εταιρεία (LTD),Πληροφορική
4,IT Support Engineer - NewCytech Business Solut...,148796,Λευκωσία,31/08/2021,27/09/2021,103,Πλήρης,Πληροφορική,Ιδιωτική Εταιρεία (LTD),Πληροφορική
...,...,...,...,...,...,...,...,...,...,...
995,Εκπρόσωπος Πωλήσεων (Sales Representative),147568,Λεμεσός,09/09/2021,18/09/2021,145,Μερική,Λιανικό Εμπόριο,Ιδιωτική Εταιρεία (LTD),Πωλητές - Καταστήματα
996,"Ταξιθέτες/τριες (K Cineplex, Λεμεσός)",147617,Λεμεσός,09/09/2021,18/09/2021,223,Πλήρη ή Μερική,Χώροι αναψυχής,Ιδιωτική Εταιρεία (LTD),Ταξιθέτες
997,"Ταμίες (K Cineplex, Λεμεσός)",147612,Λεμεσός,09/09/2021,18/09/2021,219,Πλήρη ή Μερική,Χώροι αναψυχής,Ιδιωτική Εταιρεία (LTD),Ταμίες
998,Barista,147635,Λάρνακα,09/09/2021,18/09/2021,202,Πλήρης,Εστιατόρια / Καφετέριες,Ιδιωτική Εταιρεία (LTD),"Τουριστικές Σπουδές, Υπάλληλοι HORECA, Baristas"


In [182]:
compression_opts = dict(method='zip',
                        archive_name='jobs.csv')  
df.to_csv(r'..\Data\jobs.zip', index=False,
          compression=compression_opts)

## Main

In [6]:
#OPEN THE SITE

url = "https://www.ergodotisi.com/"
driver=opensite(url)
#--------------------------------------------------------------------

# click Αναζητηση Εργασιας

sleep(random.uniform(0, 1))
SearchButton= WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[name='ctl00$ContentplaceholderMenu$ASPxButton1']")))
SearchButton.click()
#--------------------------------------------------------------------

# put all job in the page

sleep(random.uniform(2, 3))
start = time.time()

All_Jobs_to_me(driver)
#--------------------------------------------------------------------

#sometimes appear the first message with  οχι !!

for check in range(3):
    try:
        driverjob.find_element_by_xpath("//*[contains(text(), 'Οχι')]").click()
        break
    except:
        continue
        sleep(random.uniform(0,0.5))
#--------------------------------------------------------------------


#How many jobs have the site
tableJobs=driver.find_element_by_id("ctl00_ContentplaceholderSubMenu_ASPxCallbackPanel1_ASPxDataView2_ICell")
ListOfJobs=tableJobs.find_elements_by_tag_name("tr")
print(len(ListOfJobs))

# Get the links of jobs
linksjobs=[]
for job in tqdm(ListOfJobs):
    linksjobs.append(job.find_element_by_tag_name("a").get_attribute('href'))

Ergodotisi.com - Θέσεις εργασίας στην Κύπρο
Ad Found

Total Ads: 7
tries 1
tries 2
tries 3
tries 4


  0%|          | 0/1000 [00:00<?, ?it/s]

tries 5
1000


100%|██████████| 1000/1000 [00:12<00:00, 81.55it/s]


In [ ]:
# extract specific data from each url

for Link in tqdm(linksjobs):
    try:
        job_driver=opensite(Link)
        titleofjob, refernce_number, city, expire_date, views=job_analysis(job_driver)
        job_driver.quit()
        sleep(random.uniform(2,3))
    except:
        pass
    

In [8]:
(linksjobs)

['https://www.ergodotisi.com/JobDetails/148792/MESITIS-REAL-ESTATE-AGENTS-LTD',
 'https://www.ergodotisi.com/JobDetails/148797/Logicom-Public-Ltd',
 'https://www.ergodotisi.com/JobDetails/148790/MSEA-MANAGEMENT-LIMITED',
 'https://www.ergodotisi.com/JobDetails/148788/3CX',
 'https://www.ergodotisi.com/JobDetails/148796/Logicom-Public-Ltd',
 'https://www.ergodotisi.com/JobDetails/148778/D.ELLINAS-HOLDING-LTD',
 'https://www.ergodotisi.com/JobDetails/148784/3CX',
 'https://www.ergodotisi.com/JobDetails/148795/Logicom-Public-Ltd',
 'https://www.ergodotisi.com/JobDetails/148782/3CX',
 'https://www.ergodotisi.com/JobDetails/148764/DOMETICA-LTD',
 'https://www.ergodotisi.com/JobDetails/148783/3CX',
 'https://www.ergodotisi.com/JobDetails/148789/To-Tavernaki',
 'https://www.ergodotisi.com/JobDetails/148769/3CX',
 'https://www.ergodotisi.com/JobDetails/148773/3CX',
 'https://www.ergodotisi.com/JobDetails/148768/Brainrocket',
 'https://www.ergodotisi.com/JobDetails/148791/Cypronetwork-Consultan

In [9]:
driverjob=opensite(linksjobs[19])

Συντηρητής/Συντηρήτρια Συσκευών Draught - Photos Photiades Distributors Ltd
Ad Found

Total Ads: 6


In [10]:
# the actually job in the site
thejob=driverjob.find_element_by_id("ctl00_ContentplaceholderSubMenu_PanelLiveContent")
info_of_job=thejob.find_elements_by_tag_name("a")

titleofjob=thejob.find_element_by_tag_name("h1").text
refernce_number=int(info_of_job[0].text[10:])
city=info_of_job[1].text
date_posted=info_of_job[2].text
expire_date=info_of_job[4].text
views=int(info_of_job[5].text)

NameError: name 'driverjob' is not defined

In [11]:
date_posted

'αναρτήθηκε σήμερα'

In [50]:
date_posted=info_of_job[2].text
if date_posted=='αναρτήθηκε σήμερα':    
    # get the date and time now
    now = datetime.now()
    date_posted = now.strftime("%d/%m/%Y")
elif date_posted=='αναρτήθηκε χθές':
    # get the date and time now
    now = datetime.now()
    date_posted=(now - timedelta(hours=24)).strftime("%d/%m/%Y")
else:
    try:
        Posted_days=int(re.findall(r'[0-9]+', date_posted)[0])
        # get the date and time now
        now = datetime.now()
        date_posted=(now + timedelta(days=Posted_days)).strftime("%d/%m/%Y")
    except:
        pass


IndentationError: expected an indented block (<ipython-input-50-05f49dd34ab6>, line 17)

In [16]:
expire_date=info_of_job[4].text
try:

    expire_days=int(re.findall(r'[0-9]+', expire_date)[0])
    # get the date and time now
    now = datetime.now()
    expire_date=(now + timedelta(days=expire_days)).strftime("%d/%m/%Y")
except:
    pass

λήγει σε 31 ημέρες
31


In [17]:
print(f"titleofjob={titleofjob}\nrefernce_number={refernce_number}\ncity={city}\ndate_posted={date_posted}\nexpire_date={expire_date}\nviews={views}")

titleofjob=Συντηρητής/Συντηρήτρια Συσκευών Draught
refernce_number=148766
city=Λευκωσία
date_posted=27/08/2021
expire_date=27/09/2021
views=75


In [18]:
driverjob.quit()

In [19]:
views

75

In [408]:
now = datetime.now()


In [407]:
now.timedelta(days=1)

AttributeError: 'datetime.datetime' object has no attribute 'timedelta'

In [416]:
today = datetime.now()

yesterday = today-datetime.timedelta(days=1)

AttributeError: type object 'datetime.datetime' has no attribute 'timedelta'

In [423]:
print (datetime.now() - timedelta(days=1))  # Here, I am adding a negative timedelta


2021-08-25 13:54:11.223501


In [431]:
now = datetime.now()
yesterday=(now - timedelta(hours=24)).strftime("%d/%m/%Y")

In [432]:
yesterday

'25/08/2021'

In [457]:
expire_date= info_of_job[4].text
print(expire_date)

λήγει σε 30 ημέρες


In [459]:
re.findall('[0-9]', expire_date)


['3', '0']

In [473]:
re.findall(r'\d+', expire_date)

['30']

In [476]:
int(re.findall(r'[0-9]+', expire_date)[0])

30

In [172]:
def job_analysis(driverjob):
    # the actually job in the site
    thejob=driverjob.find_element_by_id("ctl00_ContentplaceholderSubMenu_PanelLiveContent")
    info_of_job=thejob.find_elements_by_tag_name("a")
    specific=thejob.find_elements_by_class_name('col-md-6')
    specificjob=specific[0].find_elements_by_tag_name('p')
    specificjob1=specific[1].find_elements_by_tag_name('p')
    
    #------titleofjob------    
    titleofjob=thejob.find_element_by_tag_name("h1").text  
    #----------------------
    
    
    #----refernce_number---    
    refernce_number=int(info_of_job[0].text[10:])    
    #----------------------
    
    
    #---------city---------    
    city=info_of_job[1].text    
    #----------------------
    
    #------date_posted-----    
    date_posted=info_of_job[2].text
    if date_posted=='αναρτήθηκε σήμερα':    
        # get the date and time now
        now = datetime.now()
        date_posted = now.strftime("%d/%m/%Y")
    elif date_posted=='αναρτήθηκε χθές':
        # get the date and time now
        now = datetime.now()
        date_posted=(now - timedelta(hours=24)).strftime("%d/%m/%Y")
    else:
        try:
            Posted_days=int(re.findall(r'[0-9]+', date_posted)[0])
            # get the date and time now
            now = datetime.now()
            date_posted=(now + timedelta(days=Posted_days)).strftime("%d/%m/%Y")
        except:
            pass
    #----------------------
    
    #------expire_date-----   
    expire_date=info_of_job[4].text
    try:

        expire_days=int(re.findall(r'[0-9]+', expire_date)[0])
        # get the date and time now
        now = datetime.now()
        expire_date=(now + timedelta(days=expire_days)).strftime("%d/%m/%Y")
    except:
        pass
    #----------------------
    
    #---------views--------   
    views=int(info_of_job[5].text)
    #----------------------
    
    #------type_of_job-----   
    type_of_employment=specificjob1[0].text
    try:
        if type_of_employment[:18]=='ΤΥΠΟΣ ΑΠΑΣΧΟΛΗΣΗΣ:':
            type_of_employment=type_of_employment[19:]
    except:
        pass
    #----------------------
    
    #---------employment_sector--------   
    employment_sector=specificjob1[1].text
    try:
        if employment_sector[:19]=='ΚΛΑΔΟΣ ΑΠΑΣΧΟΛΗΣΗΣ:':
            employment_sector=employment_sector[20:]
    except:
        pass
    #----------------------------------
    
    #-------------publisher------------   
    publisher=specificjob[1].text
    try:
        if publisher[:18]=='ΔΗΜΟΣΙΟΠΟΙΗΣΗ ΑΠΟ:':
            publisher=publisher[19:]
    except:
        pass
    #----------------------------------
    
    #-------------category-------------   
    category=specificjob[0].text
    try:
        if category[:22]=='ΚΑΤΗΓΟΡΙΑ ΑΠΑΣΧΟΛΗΣΗΣ:':
            category=category[23:]
    except:
        pass
    #----------------------------------
    
    
    
    print(f"titleofjob={titleofjob}\nrefernce_number={refernce_number}\ncity={city}\ndate_posted={date_posted}\nexpire_date={expire_date}\nviews={views}\ntype_of_employment={type_of_employment}\nemployment_sector={employment_sector}\npublisher={publisher}\ncategory={category}")

    return (titleofjob, refernce_number, city, date_posted, expire_date, views, type_of_employment, employment_sector, publisher, category)


In [173]:
# linksjobs

In [174]:
driver=opensite(linksjobs[1])
job_analysis(driver)

Systems Engineer - NewCytech Business Solutions - Nicosia & Limassol - Logicom Public Ltd
titleofjob=Systems Engineer - NewCytech Business Solutions - Nicosia & Limassol
refernce_number=148797
city=Λεμεσός, Λευκωσία
date_posted=31/08/2021
expire_date=27/09/2021
views=155
type_of_employment=Πλήρης
employment_sector=Πληροφορική
publisher=Δημόσια Εταιρεία (PLC)
category=Πληροφορική


('Systems Engineer - NewCytech Business Solutions - Nicosia & Limassol',
 148797,
 'Λεμεσός, Λευκωσία',
 '31/08/2021',
 '27/09/2021',
 155,
 'Πλήρης',
 'Πληροφορική',
 'Δημόσια Εταιρεία (PLC)',
 'Πληροφορική')

In [118]:
thejob=driver.find_element_by_id("ctl00_ContentplaceholderSubMenu_PanelLiveContent")

In [119]:
specific=thejob.find_elements_by_class_name('col-md-6')

In [61]:
specificjobs1=specific[1].find_elements_by_tag_name('p')

In [62]:
type_of_employment=specificjobs1[0].text
try:
    if type_of_employment[:18]=='ΤΥΠΟΣ ΑΠΑΣΧΟΛΗΣΗΣ:':
        type_of_employment=type_of_employment[19:]
except:
    pass

In [63]:
type_of_employment

'Πλήρης'

In [101]:
employment_sector=specificjobs1[1].text

In [103]:
employment_sector=specificjobs1[1].text
try:
    if type_of_employment[:19]=='ΚΛΑΔΟΣ ΑΠΑΣΧΟΛΗΣΗΣ:':
        employment_sector=employment_sector[20:]
except:
    pass

In [104]:
employment_sector

'Πληροφορική'

In [141]:
specificjob=specific[0].find_elements_by_tag_name('p')

In [150]:
publisher=specificjob[1].text
try:
    if publisher[:18]=='ΔΗΜΟΣΙΟΠΟΙΗΣΗ ΑΠΟ:':
        publisher=publisher[19:]
except:
    pass

In [151]:
publisher

'Δημόσια Εταιρεία (PLC)'

In [164]:
category=specificjob[0].text
try:
    if category[:22]=='ΚΑΤΗΓΟΡΙΑ ΑΠΑΣΧΟΛΗΣΗΣ:':
        category=category[23:]
except:
    pass

In [165]:
category

'Πληροφορική'